# Sambhav Agrawal 19264 Data Science and Engineering

In [1]:
import numpy as np
import pandas as pd 
from sklearn.utils.multiclass import unique_labels
import os

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [3]:
from keras import Sequential
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from keras.utils import to_categorical

In [4]:
from keras.datasets import cifar10

Turn on the internet to download the dataset

In [5]:
(X_train,y_train),(X_test,y_test)=cifar10.load_data()



In [6]:
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=.3)

In [7]:
print((X_train.shape,y_train.shape))
print((X_val.shape,y_val.shape))
print((X_test.shape,y_test.shape))

Doing one hot encoding because we have 10 class labels so we should be getting the shape of y equal to 10 and not 1

In [8]:
#One hot encode the labels.Since we have 10 classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 10

y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)


print((X_train.shape,y_train.shape))
print((X_val.shape,y_val.shape))
print((X_test.shape,y_test.shape))

We could do data augmentation to increase the dataset for better prediction results

In [9]:
train_generator = ImageDataGenerator(
                                    rotation_range=1, 
                                    horizontal_flip=True,
                                    zoom_range=.1 )

val_generator = ImageDataGenerator(
                                    rotation_range=1, 
                                    horizontal_flip=True,
                                    zoom_range=.1)

test_generator = ImageDataGenerator(
                                    rotation_range=1, 
                                    horizontal_flip= True,
                                    zoom_range=.1) 




In [10]:
train_generator.fit(X_train)
val_generator.fit(X_val)
test_generator.fit(X_test)

In [11]:
lrr= ReduceLROnPlateau(
                       monitor='val_acc', 
                       factor=.01, 
                       patience=3 ,
                         min_lr=1e-5)  

Building the Model

In [12]:
base_model = VGG16(include_top=False,weights='imagenet',input_shape=(32,32,3),classes=y_train.shape[1])


In [13]:
model= Sequential()
model.add(base_model) 
model.add(Flatten()) 

In [14]:
model.summary()

So after the flatten layer we see the output to be a vector of (512,1).This will help in deciding the number of neurons in the dense layer following the flatten layer.

## Adding Dense layers with activation and batch normalization


In [15]:
model.add(Dense(1024,activation=('relu'),input_dim=512))
model.add(Dense(512,activation=('relu'))) 
model.add(Dense(256,activation=('relu'))) 
model.add(Dense(128,activation=('relu')))
model.add(Dense(10,activation=('softmax'))) 

In [16]:
model.summary()

In [17]:
batch_size= 100
epochs=20

In [18]:
learn_rate=.001

sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)
adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)


In [19]:
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])


In [20]:
model.fit_generator(train_generator.flow(X_train,y_train,batch_size=batch_size),
                      epochs=epochs,
                      steps_per_epoch=X_train.shape[0]//batch_size,
                      validation_data=val_generator.flow(X_val,y_val,batch_size=batch_size),validation_steps=250,
                      callbacks=[lrr],verbose=1)

I have taken only 20 epochs and it is giving a really good accuaracy.It was taking a very long time to train that is why I took a small number of epochs.

1. Let's now begin to evaluate our model.

In [21]:
f,ax=plt.subplots(2,1)

#Assign the first subplot to graph training loss and validation loss
ax[0].plot(model.history.history['loss'],color='b',label='Training Loss')
ax[0].plot(model.history.history['val_loss'],color='r',label='Validation Loss')

#Next lets plot the training accuracy and validation accuracy
ax[1].plot(model.history.history['acc'],color='b',label='Training  Accuracy')
ax[1].plot(model.history.history['val_acc'],color='r',label='Validation Accuracy')

In [22]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

   
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    fig, ax = plt.subplots(figsize=(7,7))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

Now that we have our code for the confusion matrix, let's make predictions on the test set and see how this model has performed

In [23]:
y_pred=model.predict_classes(X_test)
y_true=np.argmax(y_test,axis=1)
confusion_mtx=confusion_matrix(y_true,y_pred)


In [24]:
class_names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [25]:
# Plot non-normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes=class_names,
                      title='Confusion matrix, without normalization')


In [26]:
# Plot normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')
# plt.show()